In [9]:
import pandas as pd

df = pd.read_csv('train.csv')

/Users/federicoungarelli/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (30) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [10]:
import pandas as pd

test_df = pd.read_csv('test.csv')

In [2]:
df

,id,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,...,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,22267382,Modern and Cozy Large Studio in Brooklyn,Modern large studio with new amenities and app...,Our place is a little quiet sanctuary in the h...,Modern large studio with new amenities and app...,none,"BAM, Barclays, Brooklyn City Point, Fort Green...",NaN,"Subway: 2,3,4,5,A,C,B,Q,G",Washer/Dryer Dishwasher Internet Gym Roof Top ...,...,10.0,10.0,10.0,t,f,flexible,f,f,1,0.59
1,2473861,Royal Harlem TRIPLEX Home 5 Beds,Harlem is back and so gorgeous! Visit and expl...,Harlem is back and so gorgeous! Visit and expl...,Harlem is back and so gorgeous! Visit and expl...,none,HARLEM is a piece of real NY history overflowi...,HARLEM RESTAURANTS Red Rooster Harlem -- excel...,PUBLIC TRANSPORTATION: Conveniently near all p...,The WHOLE ENTIRE HOUSE,...,9.0,9.0,9.0,t,f,moderate,f,f,3,2.47
2,25079703,Sunny East Village Studio,"Clean, hip and well designed sun drenched East...",This is a rare East Village studio with it's h...,"Clean, hip and well designed sun drenched East...",none,East Village is one of the last remaining neig...,NaN,NaN,You'll have access to the entire space - it's ...,...,10.0,10.0,10.0,f,f,moderate,f,f,1,0.89
3,9342478,"Beautiful, airy, light-filled room","Private, spacious, comfortable room in 2-bed f...","Big closet, two big windows, tall ceiling and ...","Private, spacious, comfortable room in 2-bed f...",none,One block from Morgan L stop. Super cool area....,NaN,NaN,NaN,...,NaN,NaN,NaN,f,f,flexible,f,f,1,NaN
4,4866426,Private Room in Prime Brooklyn Spot,"Comfy, quiet and big private room in a three b...",This big old apartment that we love and take c...,"Comfy, quiet and big private room in a three b...",none,I absolutely love this neighborhood - right at...,Just a note about the space: The window in you...,Super convenient to almost all subway lines. A...,Your room has a very comfortable queen sized b...,...,10.0,10.0,10.0,f,f,flexible,f,f,1,3.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33533,3958032,Private Room Amazing View in EV,"Private room in cozy, sun-drenched & plant fil...",NaN,"Private room in cozy, sun-drenched & plant fil...",none,NaN,NaN,NaN,NaN,...,9.0,9.0,8.0,f,f,strict_14_with_grace_period,f,f,1,0.44
33534,7987209,Floor 35th in the heart of New York,This is a beautiful four bedroom and three bat...,This is a Duplex Apartment (2 floors) Main Flo...,This is a beautiful four bedroom and three bat...,none,LOCATION LOCATION LOCATION. Walk to Time Squar...,NaN,Apartment is located at 70 W 45th Street (betw...,Access to the unit is through elevators.,...,10.0,10.0,9.0,f,f,strict_14_with_grace_period,f,f,1,2.28
33535,796128,Cozy room for rent in duplex house,NaN,Room for rent in duplex house located in Astor...,Room for rent in duplex house located in Astor...,none,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,f,f,flexible,f,f,1,NaN
33536,17281121,"Beautiful 1 bedroom apart, Washington Heights",Very spacious bright beautiful apartment in Wa...,"Very spacious living room, bright kitchen with...",Very spacious bright beautiful apartment in Wa...,none,Very close to Times Square but a quiet residen...,NaN,Very Close to subway lines A and 1 trains. Les...,"Living room, bedroom, shared kitchen and bathr...",...,10.0,10.0,10.0,f,f,moderate,f,f,3,0.10


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33538 entries, 0 to 33537
Data columns (total 65 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                33538 non-null  int64  
 1   name                              33526 non-null  object 
 2   summary                           32266 non-null  object 
 3   space                             23038 non-null  object 
 4   description                       33230 non-null  object 
 5   experiences_offered               33538 non-null  object 
 6   neighborhood_overview             19948 non-null  object 
 7   notes                             13447 non-null  object 
 8   transit                           20796 non-null  object 
 9   access                            19304 non-null  object 
 10  interaction                       18670 non-null  object 
 11  house_rules                       19983 non-null  object 
 12  host

In [12]:
from math import isnan

def to_float(s):
    try:
        ret = float(s)
    except:
        ret = -1
    if isnan(ret):
        ret = -1
    return ret

def to_int(s):
    try:
        ret = int(s)
    except:
        ret = -1
    return ret

categories = [x for x in list(set(df['neighbourhood_group_cleansed'])) if type(x) is str]
print(len(categories))

def create_feature(row):
    guest_num = to_int(row.guests_included)
    guest_cap = to_int(row.extra_people)
    bedrooms = to_int(row.bedrooms)
    bathrooms = to_int(row.bathrooms)
    review = to_float(row.review_scores_rating)
    one_hot = [int(row.neighbourhood_group_cleansed == category) for category in categories]
    
    return [guest_num, guest_cap, bedrooms, bathrooms, review] + one_hot

train_X, train_y = [], []
for (idx, row) in df.iterrows():
    price = float(row.price)
    feature = create_feature(row)

    train_X.append(feature)
    train_y.append(price)

5


In [4]:
from sklearn.ensemble import RandomForestRegressor
rnd_forest = RandomForestRegressor()
rnd_forest.fit(train_X, train_y)

RandomForestRegressor()

In [7]:
test_ids, test_X = [], []
for (idx, row) in test_df.iterrows():
    feature = create_feature(row)
    test_ids.append(row.id)
    test_X.append(feature)
test_y = rnd_forest.predict(test_X)

In [8]:
output_df = pd.DataFrame()
output_df['Id'] = test_ids
output_df['Predicted'] = test_y
output_df.to_csv('random_forest_baseline.csv', index = False)

In [9]:
from sklearn.svm import SVR
svr = SVR()
svr.fit(train_X, train_y)

SVR()

In [12]:
test_ids, test_X = [], []
for (idx, row) in test_df.iterrows():
    feature = create_feature(row)
    test_ids.append(row.id)
    test_X.append(feature)
test_y = svr.predict(test_X)

In [13]:
output_df = pd.DataFrame()
output_df['Id'] = test_ids
output_df['Predicted'] = test_y
output_df.to_csv('SVR_baseline.csv', index = False)

In [15]:
from sklearn.decomposition import PCA
pca = PCA(0.95)
pca.fit(train_X)

PCA(n_components=0.95)

In [24]:
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, RandomizedSearchCV
from scipy.stats import uniform, truncnorm
import numpy as np
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(40, 1000, num = 20)]
# Number of features to consider at every split
max_features = ['auto','sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [19]:
reg = RandomizedSearchCV(estimator = rnd_forest, param_distributions = random_grid, n_iter = 50, cv = 3, verbose = 2, random_state = 16, n_jobs = -1)
model = reg.fit(train_X, train_y)
# print winning set of hyperparameters
model.best_params_

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 10.4min finished


{'n_estimators': 444,
 'min_samples_split': 10,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 10,
 'bootstrap': False}

In [21]:
opt_rnd_reg = RandomForestRegressor(n_estimators= 444,min_samples_split=10, min_samples_leaf=1,
                                     max_features= 'sqrt', max_depth= 10, bootstrap=False )
opt_rnd_reg.fit(train_X, train_y)

RandomForestRegressor(bootstrap=False, max_depth=10, max_features='sqrt',
                      min_samples_split=10, n_estimators=444)

In [25]:
test_ids, test_X = [], []
for (idx, row) in test_df.iterrows():
    feature = create_feature(row)
    test_ids.append(row.id)
    test_X.append(feature)
test_y = opt_rnd_reg.predict(test_X)

In [26]:
output_df = pd.DataFrame()
output_df['Id'] = test_ids
output_df['Predicted'] = test_y
output_df.to_csv('opt_random_forest_baseline.csv', index = False)

In [29]:
from sklearn.linear_model import Ridge
rid = Ridge()
rid.fit(train_X, train_y)

Ridge()

In [30]:
test_ids, test_X = [], []
for (idx, row) in test_df.iterrows():
    feature = create_feature(row)
    test_ids.append(row.id)
    test_X.append(feature)
test_y = rid.predict(test_X)

In [31]:
output_df = pd.DataFrame()
output_df['Id'] = test_ids
output_df['Predicted'] = test_y
output_df.to_csv('ridge.csv', index = False)

In [36]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures()

In [13]:
categories = [x for x in list(set(df['neighbourhood_group_cleansed'])) if type(x) is str]
inst_bookable = [x for x in list(set(df['instant_bookable'])) if type(x) is str]

def create_feature_2(row):
    guest_num = to_int(row.guests_included)
    guest_cap = to_int(row.extra_people)
    bedrooms = to_int(row.bedrooms)
    bathrooms = to_int(row.bathrooms)
    minimum_nights = to_int(row.minimum_nights)
    number_of_reviews = to_int(row.number_of_reviews)
    reviews_per_month = to_int(row.reviews_per_month)
    review = to_float(row.review_scores_rating)
    one_hot = [int(row.neighbourhood_group_cleansed == category) for category in categories]
    one_hot_2 = [int(row.instant_bookable == s) for s in inst_bookable]
    return [guest_num, guest_cap, bedrooms, bathrooms, review, number_of_reviews, minimum_nights, reviews_per_month] + one_hot + one_hot_2

train_X, train_y = [], []
for (idx, row) in df.iterrows():
    price = float(row.price)
    feature = create_feature_2(row)

    train_X.append(feature)
    train_y.append(price)

In [70]:
opt_rnd_reg.fit(train_X, train_y)

RandomForestRegressor(bootstrap=False, max_depth=10, max_features='sqrt',
                      min_samples_split=10, n_estimators=444)

In [71]:
test_ids, test_X = [], []
for (idx, row) in test_df.iterrows():
    feature = create_feature_2(row)
    test_ids.append(row.id)
    test_X.append(feature)
test_y = opt_rnd_reg.predict(test_X)

In [72]:
output_df = pd.DataFrame()
output_df['Id'] = test_ids
output_df['Predicted'] = test_y
output_df.to_csv('opt_random_forest_2_baseline.csv', index = False)

In [4]:
pip install xgboost

     |████████████████████████████████| 1.8 MB 180 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [22]:
from xgboost import XGBRegressor
model = XGBRegressor()

In [25]:
# Define the search space
param_grid = { 
    # Percentage of columns to be randomly samples for each tree.
    "colsample_bytree": [ 0.3, 0.5 , 0.8 ],
    # reg_alpha provides l1 regularization to the weight, higher values result in more conservative models
    "reg_alpha": [0, 0.5, 1, 5],
    # reg_lambda provides l2 regularization to the weight, higher values result in more conservative models
    "reg_lambda": [0, 0.5, 1, 5]
    }
# Set up score
scoring = ['recall']
# Set up the k-fold cross-validation
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

In [31]:
grid_search = GridSearchCV(estimator=model, 
                           param_grid=param_grid, 
                           scoring= 'neg_mean_squared_error',  
                           n_jobs=-1, 
                           cv=kfold, 
                           verbose=0)
# Fit grid search
grid_result = grid_search.fit(train_X, train_y)
# Print grid search summary
grid_result.best_parameters

/Users/federicoungarelli/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"


AttributeError: 'GridSearchCV' object has no attribute 'best_parameters'

In [32]:
grid_result.best_params_

{'colsample_bytree': 0.5, 'reg_alpha': 5, 'reg_lambda': 5}

In [36]:
opt_xgb = XGBRegressor(colsample_bytree = 0.5, reg_alpha= 5, reg_lambda= 5)
opt_xgb.fit(train_X, train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.5, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [39]:
test_ids, test_X = [], []
for (idx, row) in test_df.iterrows():
    feature = create_feature_2(row)
    test_ids.append(row.id)
    test_X.append(feature)
test_y = opt_xgb.predict(test_X)

In [40]:
output_df = pd.DataFrame()
output_df['Id'] = test_ids
output_df['Predicted'] = test_y
output_df.to_csv('xgboost_2_baseline.csv', index = False)

NameError: name 'sklearn' is not defined

In [1]:
import lightgbm as ltb
lgb_model = ltb.LGBMRegressor()

params = {'learning_rate': uniform(0, 1),
          'n_estimators': sp_randint(200, 1500),
          'num_leaves': sp_randint(20, 200),
          'max_depth': sp_randint(2, 15),
          'min_child_weight': uniform(0, 2),
          'colsample_bytree': uniform(0, 1),
         }
lgb_random = RandomizedSearchCV(lgb_model, param_distributions=params, n_iter=10, cv=3, random_state=42, 
                                scoring='neg_root_mean_squared_error', verbose=10, return_train_score=True)
lgb_random = lgb_random.fit(X_train, y_train)

best_params = lgb_random.best_params_
print(best_params)

OSError: dlopen(/Users/federicoungarelli/opt/anaconda3/lib/python3.8/site-packages/lightgbm/lib_lightgbm.so, 6): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /Users/federicoungarelli/opt/anaconda3/lib/python3.8/site-packages/lightgbm/lib_lightgbm.so
  Reason: image not found

In [64]:
df2 = df.loc[:, ['guests_included',  'extra_people', 'bedrooms', 'bathrooms', 'minimum_nights', 'review_scores_rating', 
                'neighbourhood_group_cleansed', 'instant_bookable', 'price']]

In [65]:
df2 = df2.dropna()

In [61]:
train_X, train_y = [], []
for (idx, row) in df2.iterrows():
    price = float(row.price)
    feature = create_feature_2(row)

    train_X.append(feature)
    train_y.append(price)

In [62]:
test_ids, test_X = [], []
for (idx, row) in test_df.iterrows():
    feature = create_feature_2(row)
    test_ids.append(row.id)
    test_X.append(feature)
test_y = opt_rnd_reg.predict(test_X)

In [63]:
output_df = pd.DataFrame()
output_df['Id'] = test_ids
output_df['Predicted'] = test_y
output_df.to_csv('opt_random_forest_3_baseline.csv', index = False)